In [ ]:
import functions as f
import pandas as pd

url_1 = '../data/raw/SYB67_176_202411_Tourist-Visitors Arrival and Expenditure.csv'
df_1 = pd.read_csv(url_1)

url_2 = '../data/raw/SYB67_285_202411_Research and Development Expenditure and Staff.csv'
df_2 = pd.read_csv(url_2)

url_3 = '../data/raw/SYB67_328_202411_Intentional homicides and other crimes.csv'
df_3 = pd.read_csv(url_3)

url_4 = '../data/raw/SYB67_329_202411_Labour Force and Unemployment.csv'
df_4 = pd.read_csv(url_4)

#Function to clean rows and columns
df_1 = f.clean_row_column(df_1,url_1)
df_2 = f.clean_row_column(df_2,url_2)
df_3 = f.clean_row_column(df_3,url_3)
df_4 = f.clean_row_column(df_4,url_4)

#Function to insert column type
df_1 = f.insert_column(df_1,"Type","Tourism")
df_2 = f.insert_column(df_2,"Type","R&D")
df_3 = f.insert_column(df_3,"Type","Crime")
df_4 = f.insert_column(df_4,"Type","Labour & Unemployment")

#Concat all the dataframes.
df = pd.concat([df_1,df_2,df_3,df_4], axis=0)

#Fill N/A values.
df = df.fillna("N/A")

#Format column names
df = df.rename(columns={col: col.rstrip().replace(" ", "_").lower() for col in df.columns})

#Drop irrelevant columns
df = df.drop(columns=['source','tourism_arrivals_series_type_footnote'])

#Replace N/A values for tourism_arrivals_series_type column
#df['tourism_arrivals_series_type'] = df['tourism_arrivals_series_type'].replace("N/A","TE")
# #Replace N/A values for tourism_arrivals_series_type column
df.loc[df['type'] == 'Tourism', 'tourism_arrivals_series_type'] = df.loc[df['type'] == 'Tourism', 'tourism_arrivals_series_type'].replace("N/A", "TE")

#Replacing strings to float in value column
df['value']=df['value'].astype(str).str.replace(',','').astype(float)

# Apply the function to create a new column 'Continent'
df['continent'] = df['region/country/area'].apply(f.get_continent)
df


In [ ]:
ue = ['Unemployment rate - Male', 'Unemployment rate - Female']
df_le = df[(df['type']== 'Labour & Unemployment') & (df['series'].isin(ue)) ]
df_le



### HIGHEST UNEMPLOYEMENT RATE BY GENNDER & CONTINENT

In [ ]:
import matplotlib.pyplot as plt
gender_unemployment = df_le.groupby(['continent', 'series'])['value'].mean().unstack()
print(gender_unemployment)

# unStacked bar chart using actual unemployment rates
gender_unemployment.plot(kind='bar', stacked=False, color=['blue', 'red'])

plt.title('Unemployment Rate by Gender and Continent')
plt.xlabel('Continent')
plt.ylabel('Unemployment Rate (%)')
plt.legend(title='Gender')
plt.show()

### AFRICAN COUNTRIES WITH THE HIGHEST UNEMPLOYMENT RATE

In [ ]:
# Filter dataset to include only African countries
africa_df = df_le[df_le['continent'] == 'Africa']

# Group by country and unemployment series, then find the mean unemployment rate
africa_unemployment = africa_df.groupby(['region/country/area', 'series'])['value'].mean().unstack()

# Sort countries by highest total unemployment rate (if both male and female exist)
africa_unemployment['Total Unemployment'] = africa_unemployment.mean(axis=1)
top_countries = africa_unemployment.sort_values(by='Total Unemployment', ascending=False)
(top_countries.head(10))

# top 10 African countries with highest unemployment rates
top_countries.head(10).drop(columns=['Total Unemployment']).plot(kind='bar', stacked=True, color=['purple', 'pink'])

plt.title('Top 10 African Countries with Highest Unemployment Rates')
plt.xlabel('Country')
plt.ylabel('Unemployment Rate (%)')
plt.legend(title='Gender')
plt.xticks(rotation=45)
plt.show()

In [ ]:
lf = ['Labour force participation - Total'] #'Labour force participation - Female']
df_le = df[(df['type']== 'Labour & Unemployment') & (df['series'].isin(lf))]
df_le

### CONTINENT WITH THE HIGHEST LABOUR FORCE PARTICIPATION

In [ ]:

employed = df_le.groupby(['continent', 'series'])['value'].mean().unstack()
print(employed)

# unStacked bar chart using actual unemployment rates
employed.plot(kind='bar', stacked=False, color=['red'])

plt.title('Labour force participation Rate by Continent')
plt.xlabel('Continent')
plt.ylabel('Employment Rate (%)')
plt.legend(title='')
plt.show()

### OCEANIAN COUNTRIES WITH THE HIGHEST LABOUR FORCE PARTICIPATION RATE

In [ ]:
# Filter dataset to include only African countries
oceania_df = df_le[df_le['continent'] == 'Oceania']

# Group by country and labour force participation series, then find the mean rate
oceania_labour = oceania_df.groupby(['region/country/area', 'series'])['value'].mean().unstack()

# Sort countries by highest labour force participation rate
top_countries_labour = oceania_labour.sort_values(by='Labour force participation - Total', ascending=False).head(10)
print(top_countries_labour)

top_countries_labour.head(10).plot(kind='bar', color='blue', legend=False)
plt.title('Oceania countries with highest labor participation rate')
plt.xlabel('country')
plt.ylabel('Labour employment Rate (%)')
#plt.legend(title=['labour force participation oceania'])
plt.xticks(rotation=45)
plt.show()
